In [ ]:
using DrWatson
@quickactivate "QAOA.jl"

In [ ]:
include("../src/QAOA.jl")

using Statistics, BenchmarkTools

using PyPlot, Printf
# PyPlot.plt.style.use("paper.mplstyle")
using PyCall
np = pyimport("numpy")
nx = pyimport("networkx")
pytime = pyimport("time");

## Scan graph size

### Python

In [ ]:
py"""
import networkx as nx
import pennylane as qml
from pennylane import qaoa
from pennylane import numpy as np
import time

def scan_graph_size(N, p, steps=128):
    graph = nx.gnp_random_graph(N, 0.5, seed=137)
    cost_h, mixer_h = qaoa.min_vertex_cover(graph, constrained=False)
    
    def qaoa_layer(gamma, alpha):
        qaoa.cost_layer(gamma, cost_h)
        qaoa.mixer_layer(alpha, mixer_h)    
    
    wires = range(N)
    depth = p   
    beta = np.array([0.5*(1-j/p) for j in range(1, p)] + [0.5/(4*p)])
    gamma = np.array([0.5*(j-1/2)/p  for j in range(1, p + 1)])
    
    optimizer = qml.GradientDescentOptimizer()
    params = np.array([beta, gamma], requires_grad=True)  
    
    def circuit(params, **kwargs):
        for w in wires:
            qml.Hadamard(wires=w)
        qml.layer(qaoa_layer, depth, params[0], params[1])     
    
    dev = qml.device("default.qubit", wires=wires)

    @qml.qnode(dev)
    def cost_function(params):
        circuit(params)
        return qml.expval(cost_h)

    start = time.time()
    for i in range(steps):
        params = optimizer.step(cost_function, params)
    end = time.time()

    return end - start

p = 5

nrange = [6, 8, 10, 12, 15, 20] # [5, 10, 15, 20]
num_averages = {6: 5, 8: 5, 10: 5, 12: 1, 15: 1, 20: 1}
data = list(map(lambda n: np.mean([scan_graph_size(n, p) for _ in range(num_averages[n])]), nrange))

np.save("../data/" + "qaoa_pennylane_p_" + str(p) + "_n_" + "_".join(map(lambda x: str(x), nrange)) + ".npy", data)
"""

### Julia

In [ ]:
function scan_graph_size(N, p)
    graph = nx.gnp_random_graph(N, 0.5, seed=137) 
    
    h = -ones(N)
    J = zeros(N, N)
    for edge in graph.edges
        h[edge[1] + 1] += 3/4.
        h[edge[2] + 1] += 3/4.
        J[(edge .+ (1, 1))...] = 3/4.
    end    
    
    problem = QAOA.Problem(p, h, J)
    beta = vcat([0.5(1-j/p) for j in 1:p-1], [0.5/(4p)])
    gamma = [0.5(j-1/2)/p  for j in 1:p]  

    t_0 = pytime.time()
    cost, params, probabilities = QAOA.optimize_parameters(problem, vcat(beta, gamma))
    t_f = pytime.time()
    return t_f - t_0
end

In [ ]:
# All runtime cases except n=20 are averaged over 5 runs!
nrange = [6, 8, 10, 12, 15, 20];
p = 5;

In [ ]:
data = map(n -> scan_graph_size(n, p), nrange)

In [ ]:
# new_data = map(n -> mean([scan_graph_size(n, p) for _ in 1:5]), nrange[1:end-1])

In [ ]:
# np.save("../data/qaoa_yao_p_" * string(p) * "_n_" * join(string.(nrange), "_") * ".npy", data)

In [ ]:
FILE = "../data/qaoa_pennylane_p_" * string(p) * "_n_" * join(string.(nrange), "_") * ".npy"
pennylane_data = np.load(FILE)

FILE = "../data/qaoa_yao_p_" * string(p) * "_n_" * join(string.(nrange), "_") * ".npy"
data = np.load(FILE)

In [ ]:
figure(figsize=(4, 3))
styles = ["o-", "s-", "^-"]
for (p, style) in zip([1, 3, 5], styles)
    FILE = "../data/qaoa_pennylane_p_" * string(p) * "_n_" * join(string.(nrange), "_") * ".npy"
    pennylane_data = np.load(FILE)

    FILE = "../data/qaoa_yao_p_" * string(p) * "_n_" * join(string.(nrange), "_") * ".npy"
    data = np.load(FILE)
    
    plot(nrange, pennylane_data ./ data, style, label=@sprintf("\$p=%i\$", p))
end
xlim(6, 20)
xticks(nrange)
xlabel("\$t\$")
ylim(0, 20)
ylabel("\$ t_{PennyLane} / t_{QAOA.jl}\$")
legend(frameon=false)
tight_layout()